# [PG01] Unsupervised anomaly detection in industrial image data with autoencoders

In this notebook we are going to develop the final projet for the EAI course held by Christian Napoli.

## Imports & Download

In [ ]:
# install the requirements
%pip install -r requirements.txt > /dev/null
# set to false if you already have the dataset
download_dataset = False 
if download_dataset:
    %cd dataset
    !bash dataset/download_dataset.sh
    %cd ..

In [11]:
import dataclasses
from src.data_module import MVTec_Dataset, MVTec_DataModule
from src.AE_simple import AE
from src.hyperparameters import Hparams
from dataclasses import asdict
import matplotlib.pyplot as plt
import wandb
import torchvision
import pytorch_lightning as pl
from pytorch_lightning.loggers.wandb import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
# reproducibility stuff
import numpy as np
import random
import torch
np.random.seed(0)
random.seed(0)
torch.cuda.manual_seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True  # Note that this Deterministic mode can have a performance impact
torch.backends.cudnn.benchmark = False
_ = pl.seed_everything(0)
wandb.require("service")

# to have a better workflow using notebook https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
# these commands allow to update the .py codes imported instead of re-importing everything every time.
%load_ext autoreload
%autoreload 2

Global seed set to 0


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# login wandb to have the online logger. It is really useful since it stores all the plots and evolution of the model
# check also https://docs.wandb.ai/guides/integrations/lightning
wandb.login()

## Dataset test

Just to have a visual feedback and test our code, we plot some samples from the training set (only not anomalous samples) and test set (normal and anomalous)

In [ ]:
hparams = asdict(Hparams())

In [ ]:
MVTec_Data = MVTec_DataModule(hparams)
# to setup it takes ~2.5 minutes
MVTec_Data.setup()

In [ ]:
# to make sure everything works we just plot a sample of our images
def plot_objects(images, 
                images_per_row, 
                border = 10, 
                pad_value = 1,
                title = 'Industrial images',
                figsize = (16,16)):
    plt.figure(figsize = figsize)
    plt.imshow(torchvision.utils.make_grid(images,images_per_row,border,pad_value=pad_value).permute(1, 2, 0))
    plt.title(title)
    plt.axis('off')

batch = iter(MVTec_Data.train_dataloader()).next()
plot_objects(batch["img"][0:40], images_per_row=8, title="Industrial images from training dataset")
batch2 = iter(MVTec_Data.val_dataloader()).next()
plot_objects(batch2["img"][0:40], images_per_row=8, title="Industrial images from validation dataset")

todo some statistics on the pixels of the training dataset to do a proper normalization

## Autoencoders

In [12]:
# this function encapsulate the stuff (logger, callbacks, parameters)
# needed to perform the pytorch lightning training for each model
def train_model(data, model, experiment_name, model_version, patience, metric_to_monitor, mode, epochs):
    logger =  WandbLogger(project=experiment_name, name = model_version, log_model=True, save_dir="logs")
    logger.experiment.watch(model, log = None, log_freq = 100000)
    # To limit overfitting and avoid much more epochs than needed to complete
    # the training, we use the early stopping regulation technique which is
    # very powerful since it controls whether there is an improvement in the
    # model or not. If there is no improvement in the model performances for a 
    # given number of epochs (patience) on a certain metric (metric_to_monitor)
    # then the training stops.
    early_stop_callback = EarlyStopping(
        monitor=metric_to_monitor, mode=mode, min_delta=0.00,
        patience=patience, verbose=True)
    checkpoint_callback = ModelCheckpoint(
        save_top_k=1, monitor=metric_to_monitor, mode=mode, dirpath="models",
        filename=experiment_name + "_" + model_version +
        "-{epoch:02d}-{avg_val_loss:.4f}", verbose=True)
    # the trainer collect all the useful informations so far for the training
    n_gpus = 1 if torch.cuda.is_available() else 0
    trainer = pl.Trainer(
        logger=logger, max_epochs=epochs, log_every_n_steps=1, gpus=n_gpus,
        callbacks=[early_stop_callback, checkpoint_callback],
        num_sanity_val_steps=0)
    trainer.fit(model, data)
    wandb.finish()
    return trainer


### CNN-AE l2 deterministic

In [13]:
ae_hparams = asdict(Hparams())
data = MVTec_DataModule(ae_hparams)
model = AE(ae_hparams)
trainer = train_model(data,model, "EAI_ANOMALY_DET", "Simple_AE_01", patience= 15, metric_to_monitor="loss", mode="min", epochs = 100)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
